# SpringSet: numerical and visual analysis

In [ ]:
import sys
sys.path.append('../')

import os
import numpy as np
import h5py
import essentia
import essentia.standard as es
from pathlib import Path

import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [10, 6]
%matplotlib inline

In [ ]:
# Setting up the directories and the sample rate

DATA_DIR = Path('../data/raw/')
MODELS_DIR = Path('../models/')
RESULTS_DIR = Path('../data/features/')
PLOTS_DIR = Path('../docs/plots/')

sample_rate = 16000
spring_dir = DATA_DIR / 'springset/' 

## Read the data

In [ ]:
# List and read the files in the dataset folder

data_dict = {}

for file in os.listdir(spring_dir):
    if file.endswith(('.h5')):
        file_path = os.path.join(spring_dir, file)
        with h5py.File(os.path.join(spring_dir, file), 'r') as f_handler:
            key = list(f_handler.keys())[0]
            data = f_handler[key]
            print(f"File: {file}")
            print(f"Keys: {key}")
            if len(data.attrs) > 0:
                print("Metadata attributes found:")
            for attr_name, attr_value in data.attrs.items():
                print(f"{attr_name}: {attr_value}")
            else:
                print("No metadata attributes found.")
            
            print(f"Type of ['{key}']: {type(data[:])}")
            print(f"Data shape: {data.shape}")
            print(f"Data type: {data.dtype}")
    
            data_dict[key] = data[:]  

            print("")

***The data is organized in the following way: `(index, samples, channels)`***

No attributes have been found in the data.

In [ ]:
# Load each part of the dataset into a variable
X_train = data_dict['Xtrain']
Y_train = data_dict['Ytrain_0']
X_test = data_dict['Xvalidation']
Y_test = data_dict['Yvalidation_0']

# Print the shape of each part of the dataset
print('Xtrain shape:', X_train.shape)
print('Ytrain shape:', Y_train.shape)
print('Xtest shape:', X_test.shape)
print('Ytest shape:', Y_test.shape)

In [ ]:
## Compute the stats of each file and the whole dataset ### FROM FILES NOT ARRAYS !!!

def compute_stats_of_file(data):
    min_val = np.min(data)
    max_val = np.max(data)
    mean_val = np.mean(data)
    std_val = np.std(data)
    return min_val, max_val, mean_val, std_val

# Iterate over each key-value pair in data_dict
for key, value in data_dict.items():
    stats = compute_stats_of_file(value)

    print(f"{key}: Min: {stats[0]:.3f}, Max: {stats[1]:.3f}, Mean: {stats[2]:.3f}, Std: {stats[3]:.3f}")

# Concatenate all arrays into a single one
all_data = np.concatenate(list(data_dict.values()))

# Compute the stats
global_stats = compute_stats_of_file(all_data)

print(f"Global: Min: {global_stats[0]:.3f}, Max: {global_stats[1]:.3f}, Mean: {global_stats[2]:.3f}, Std: {global_stats[3]:.3f}")

In [ ]:
samples_list = [100, 600, 700]

desired_samples = []

# Loop through the indices and plot the waveform for each example
for i, sample_idx in enumerate(samples_list):
    x = X_train[sample_idx, :, 0]
    y = Y_train[sample_idx, :, 0]
    desired_samples.append((x, y))

# Plot the extracted waveforms
fig, axs = plt.subplots(nrows=1, ncols=len(desired_samples), figsize=(10, 4))
for i, (x, y) in enumerate(desired_samples):
    axs[i].plot(x.squeeze(), alpha=0.7)
    axs[i].plot(y.squeeze(), alpha=0.7)
    axs[i].set_title(f'Random Sample {i + 1}')
    axs[i].set_xlabel('Time (samples)')
    axs[i].set_ylabel('Amplitude')
    axs[i].legend(['Input', 'Target'])
    axs[i].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=len(desired_samples), figsize=(10, 6))

# Setting sample rate and STFT parameters
sample_rate = 16000
frameSize = 512  # STFT window size
hopSize = 256   # Overlap is frameSize - hopSize

# Setting up essentia algorithms
spectrum = es.Spectrum()
windowing = es.Windowing(type='blackmanharris62', zeroPadding=frameSize)

amp2db = es.UnaryOperator(type='lin2db', scale=1)
pool = essentia.Pool()

for idx, (x, y) in enumerate(desired_samples):
    pool.clear()
    x = x.squeeze().astype(np.float32)  # Convert data to float32
    y = y.squeeze().astype(np.float32)


    spectrograms_x = []
    spectrograms_y = []

    for frame_x, frame_y in zip(es.FrameGenerator(x, frameSize=frameSize, hopSize=hopSize),
                                es.FrameGenerator(y, frameSize=frameSize, hopSize=hopSize)):
        x_stft = spectrum(windowing(frame_x))
        y_stft = spectrum(windowing(frame_y))
        
        pool.add('spectrum_db_x', amp2db(x_stft))
        pool.add('spectrum_db_y', amp2db(y_stft))
        

    # Plotting the X spectrogram
    img_x = axs[0, idx].imshow(pool['spectrum_db_x'].T, aspect='auto', origin='lower', interpolation='none', cmap='inferno')
    axs[0, idx].set_title(f"X - Spectrogram {idx + 1}")
    axs[0, idx].set_ylabel('Frequency (bins)')
    axs[0, idx].set_xlabel('Time (frames)')
    axs[0, idx].set_ylim(0, 500)
    plt.colorbar(img_x, ax=axs[0, idx])  # Adding colorbar

    # Plotting the Y spectrogram
    img_y = axs[1, idx].imshow(pool['spectrum_db_y'].T, aspect='auto', origin='lower', interpolation='none', cmap='inferno')
    axs[1, idx].set_title(f"Y - Spectrogram {idx + 1}")
    axs[1, idx].set_ylabel('Frequency (bins)')
    axs[1, idx].set_xlabel('Time (frames)')
    axs[1, idx].set_ylim(0, 500)
    plt.colorbar(img_y, ax=axs[1, idx])

plt.tight_layout()
plt.show()

In [ ]:
import auraloss
import torch 
import neural_audio_spring_reverb as nasr

from neural_audio_spring_reverb.data.springset import load_springset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset
batch_size = 64
train_loader, valid_loader, test_loader = load_springset(DATA_DIR, batch_size)

# Initialize the ESR loss function
esr_loss_func = auraloss.time.ESRLoss().to(device)
mrstft_loss_func = auraloss.freq.MultiResolutionSTFTLoss().to(device)

# Function to compute average ESR
def compute_average_metrics(data_loader):
    esr_total = 0.0
    mrstft_total = 0.0
    samples_count = 0

    for dry, wet in data_loader:
        dry, wet = dry.to(device), wet.to(device)
        esr_value = esr_loss_func(dry, wet)
        mrstft_value = mrstft_loss_func(dry, wet)
        esr_total += esr_value.item() * dry.size(0)
        mrstft_total += mrstft_value.item() * dry.size(0)
        samples_count += dry.size(0)

    average_esr = esr_total / samples_count
    average_mrstft = mrstft_total / samples_count
    return average_esr, average_mrstft

In [ ]:
# Naive Baseline

# Compute average ESR for the desired dataset split
avg_esr_naive, avg_mrstft_naive = compute_average_metrics(test_loader)
print("Naive Baseline")
print(f"Average ESR (Test Set): {avg_esr_naive}")
print(f"Average MRSTFT (Test Set): {avg_mrstft_naive}")

In [ ]:
# Dummy Regressor

class DummyRegressor(nn.Module):
    def __init__(self, output_shape):
        super(DummyRegressor, self).__init__()
        self.output_shape = output_shape
        
    def forward(self, x):
        # Generates a tensor with the same shape as the output, filled with random numbers
        return torch.rand(self.output_shape).to(x.device)

# [batch_size, 1, sample_length]

dummy_regressor = DummyRegressor(output_shape=(batch_size, 1, 32000)).to(device)

# Compute average ESR for the test set using the dummy regressor
avg_esr_dummy, avg_mrstft_dummy = compute_average_metrics(test_loader)
print("Dummy Regressor")
print(f"Average ESR (Test Set with Dummy Regressor): {avg_esr_dummy}")
print(f"Average MRSTFT (Test Set with Dummy Regressor): {avg_mrstft_dummy}")